<a href="https://colab.research.google.com/github/Sreyas2403/StockAnalysis_Patterns-Indicators/blob/main/Peak%26Valley_AdjDiff_Rise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nsepython yfinance

In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from scipy.signal import argrelextrema
import numpy as np

# Define the stock symbol
symbol = input("Enter stock symbol: ") + ".NS"

# Calculate the start and end dates for historical data (1 year for this example)
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=365)

# Fetch historical data using yfinance
df = yf.download(tickers=symbol, start=start_date, end=end_date, interval='1d')

# Ensure the index is a DatetimeIndex
df.index = pd.to_datetime(df.index)

# Identify peaks and valleys
df['Peaks'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=5)[0]]['Close']
df['Valleys'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=5)[0]]['Close']

# Get the current closing price
current_closing_price = df['Close'][-1]

# Find the nearest peak and valley to the current date
nearest_peak = df['Peaks'].dropna().iloc[-1]
nearest_valley = df['Valleys'].dropna().iloc[-1]

# Calculate the differences and percentages
peak_diff = current_closing_price - nearest_peak
valley_diff = current_closing_price - nearest_valley

peak_rise_percentage = (peak_diff / nearest_peak) * 100
valley_drop_percentage = (valley_diff / nearest_valley) * 100

# Print the results
print(f"Current closing price: {current_closing_price}")
print(f"Nearest peak closing price: {nearest_peak}")
print(f"Nearest valley closing price: {nearest_valley}")
print(f"Difference from nearest peak: {peak_diff}")
print(f"Percentage rise from nearest peak: {peak_rise_percentage}%")
print(f"Difference from nearest valley: {valley_diff}")
print(f"Drop percentage from nearest valley: {valley_drop_percentage}%")


Enter stock symbol: SBIN


[*********************100%%**********************]  1 of 1 completed

Current closing price: 852.5999755859375
Nearest peak closing price: 852.5999755859375
Nearest valley closing price: 775.2000122070312
Difference from nearest peak: 0.0
Percentage rise from nearest peak: 0.0%
Difference from nearest valley: 77.39996337890625
Drop percentage from nearest valley: 9.984515242530103%


In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from scipy.signal import argrelextrema
import numpy as np

# Function to fetch and resample data
def fetch_and_resample(symbol, start_date, end_date):
    df = yf.download(tickers=symbol, start=start_date, end=end_date, interval='1d')
    df.index = pd.to_datetime(df.index)
    return df

# Function to calculate peaks and valleys
def calculate_peaks_valleys(df):
    df['Peaks'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=10)[0]]['Close']
    df['Valleys'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=10)[0]]['Close']
    return df

# Function to calculate differences and percentages
def calculate_differences_peaks_valleys(df):
    peaks = df['Peaks'].dropna()
    valleys = df['Valleys'].dropna()

    # Ensure there are enough peaks and valleys to compare
    if len(peaks) < 2 or len(valleys) < 2:
        print("Not enough peaks or valleys for comparison.")
        return None

    # Find adjacent valleys for each peak
    results = []
    for i in range(len(peaks) - 1):
        peak1 = peaks.iloc[i]
        peak2 = peaks.iloc[i + 1]

        # Find valleys between peak1 and peak2
        valley = valleys[(valleys.index > peaks.index[i]) & (valleys.index < peaks.index[i + 1])]

        if not valley.empty:
            valley = valley.iloc[0]
            difference = peak1 - valley
            percentage_difference = (difference / valley) * 100
            results.append([peak1, valley, difference, percentage_difference])

    return results

# Define the stock symbol
symbol = input("Enter stock symbol: ") + ".NS"

# Calculate the start and end dates for historical data (1 year for this example)
end_date = datetime.datetime.now()
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

# Fetch historical data
df = fetch_and_resample(symbol, start_date, end_date)

# Calculate peaks and valleys
df = calculate_peaks_valleys(df)

# Calculate differences and percentages
results = calculate_differences_peaks_valleys(df)
print(results)

# Save results to an Excel file if not None
results_df = pd.DataFrame(results, columns=['Peak Closing Price', 'Adjacent Valley Closing Price', 'Difference', 'Percentage Difference'])
output_file = f'{symbol}_peaks_valleys_differences.xlsx'
results_df.to_excel(output_file, index=False)
print(f"Results have been saved to {output_file}")


Enter stock symbol: SBIN
Enter start date (YYYY-MM-DD): 2021-01-01
Enter end date (YYYY-MM-DD): 2023-12-31


[*********************100%%**********************]  1 of 1 completed

[[307.25, 275.6499938964844, 31.600006103515625, 11.463815274156133], [415.20001220703125, 328.8500061035156, 86.35000610351562, 26.258173787697697], [439.6499938964844, 412.8999938964844, 26.75, 6.478566334565344], [432.8500061035156, 420.8999938964844, 11.95001220703125, 2.839157134787277], [456.95001220703125, 406.70001220703125, 50.25, 12.355544256639993], [530.4500122070312, 460.54998779296875, 69.9000244140625, 15.17751086022929], [494.70001220703125, 446.0, 48.70001220703125, 10.919285248213285], [540.5499877929688, 440.29998779296875, 100.25, 22.768567517457676], [517.7000122070312, 444.6499938964844, 73.05001831054688, 16.428656092043735], [474.6000061035156, 434.70001220703125, 39.899993896484375, 9.17874229952437], [542.0999755859375, 511.29998779296875, 30.79998779296875, 6.023858503481916], [574.0499877929688, 519.75, 54.29998779296875, 10.447328098695287], [625.5, 574.0, 51.5, 8.97212543554007], [613.7000122070312, 527.3499755859375, 86.35003662109375, 16.374332154875024]

In [ ]:
!pip install bsedata